# Diagnostic Analytics

### Loading the cleaned dataset

In [1]:
import pandas as pd

file_name = '../assets/ObesityDataSet_BMI.parquet'
df = pd.read_parquet(file_name)
df.head()

,Gender,Age,Height,Weight,Family_history_overweight,High_caloric_food,Veggie_consumption_freq,Main_meals_count,Food_between_meals_freq,Smokes,Water_consumption,Monitors_calories,Physical_activity,Screen_time,Alcohol_consumption_freq,Transportation_mode,Obesity_level,BMI
0,Female,21,1.62,64.0,True,False,Sometimes,Three,Sometimes,False,Between 1 and 2 L,False,I do not,3-5 hours,no,Public_Transportation,Normal_Weight,24.386526
1,Female,21,1.52,56.0,True,False,Always,Three,Sometimes,True,More than 2 L,True,4 or 5 days,0-2 hours,Sometimes,Public_Transportation,Normal_Weight,24.238227
2,Male,23,1.80,77.0,True,False,Sometimes,Three,Sometimes,False,Between 1 and 2 L,False,2 or 4 days,3-5 hours,Frequently,Public_Transportation,Normal_Weight,23.765432
3,Male,27,1.80,87.0,False,False,Always,Three,Sometimes,False,Between 1 and 2 L,False,2 or 4 days,0-2 hours,Frequently,Walking,Overweight_Level_I,26.851852
4,Male,22,1.78,89.8,False,False,Sometimes,Between 1 and 2,Sometimes,False,Between 1 and 2 L,False,I do not,0-2 hours,Sometimes,Public_Transportation,Overweight_Level_II,28.342381


**Check structure:**

In [2]:
df.shape

(2111, 18)

**Check datatypes of the features:**

In [3]:
df.dtypes

Gender                       category
Age                             int64
Height                        float64
Weight                        float64
Family_history_overweight     boolean
High_caloric_food             boolean
Veggie_consumption_freq      category
Main_meals_count             category
Food_between_meals_freq      category
Smokes                        boolean
Water_consumption            category
Monitors_calories             boolean
Physical_activity            category
Screen_time                  category
Alcohol_consumption_freq     category
Transportation_mode          category
Obesity_level                category
BMI                           float64
dtype: object

**Check all the categorical columns and distinguish whether they are ordered or not:**

In [ ]:
for c in df.select_dtypes(['category']).columns:
    dt = df[c].dtype    # CategoricalDtype
    print(f'{c:25s} | ordered={dt.ordered} | categories={list(dt.categories)}')

Gender                    | ordered=False:25s | categories=['Female', 'Male']
Veggie_consumption_freq   | ordered=True:25s | categories=['Never', 'Sometimes', 'Always']
Main_meals_count          | ordered=True:25s | categories=['Between 1 and 2', 'Three', 'More than three']
Food_between_meals_freq   | ordered=True:25s | categories=['no', 'Sometimes', 'Frequently', 'Always']
Water_consumption         | ordered=True:25s | categories=['Less than a liter', 'Between 1 and 2 L', 'More than 2 L']
Physical_activity         | ordered=True:25s | categories=['I do not', '1 or 2 days', '2 or 4 days', '4 or 5 days']
Screen_time               | ordered=True:25s | categories=['0-2 hours', '3-5 hours', 'More than 5 hours']
Alcohol_consumption_freq  | ordered=True:25s | categories=['no', 'Sometimes', 'Frequently', 'Always']
Transportation_mode       | ordered=False:25s | categories=['Automobile', 'Bike', 'Motorbike', 'Public_Transportation', 'Walking']
Obesity_level             | ordered=True:25s | cat